In [1]:
from database_utils import DatabaseConnector
from data_extraction import DataExtractor
from data_cleaning import DataCleaning
from data_processing import DataProcessor
from IPython.display import display
import pandas as pd

In [2]:
creds_path = '/Users/itsanya/AiCore/MRDC/db_creds.yaml'
b_store_api_key = 'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'

db_connector_instance = DatabaseConnector(creds_path) 
data_extractor = DataExtractor(db_connector_instance) 

data_cleaner = DataCleaning()
data_processor = DataProcessor()

In [3]:
#Legacy Users
#Extract
#table_name = 'legacy_users'
#users_df = data_extractor.read_rds_table(table_name)

#View
#display(users_df)

#Clean
#fully_cleaned_users_df = data_cleaner.clean_users_df(users_df)

#View Cleaned DataFrame
#display(fully_cleaned_users_df)

#Upload to dim_users
#db_connector_instance.upload_to_db(df=fully_cleaned_users_df, table_name='dim_users')

In [4]:
#Card Data
#Extract
pdf_link = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf'
card_df = data_extractor.retrieve_pdf_data(pdf_link)

#View
#display(card_df)
#card_df.info()

#Clean
fully_cleaned_card_df = data_cleaner.clean_card_data(card_df)

#View Cleaned DataFrame
display(fully_cleaned_card_df)

#Upload to dim_card_details
db_connector_instance.upload_to_db(df=fully_cleaned_card_df, table_name='dim_card_details')

Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'


KeyboardInterrupt: 

In [ ]:
#Business Store Data
#Extract

#number of stores: - WORKS!
#number_of_stores_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores'
#data_extractor.set_api_key(b_store_api_key)
#number_of_stores = data_extractor.list_number_of_stores(number_of_stores_endpoint)

#stores data: - DOES NOT WORK...
#store_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/{store_number}'
#data_extractor.set_api_key(b_store_api_key)
#b_store_df = data_extractor.retrieve_stores_data(store_endpoint)

#View
#display(b_store_df)

#Clean
#fully_cleaned_b_store_df = data_cleaner.clean_store_data(b_store_df)

#View Cleaned DataFrame
#display(fully_cleaned_b_store_df)

#Upload to dim_store_details
#db_connector_instance.upload_to_db(df=fully_cleaned_b_store_df, table_name='dim_store_details')